<a href="https://colab.research.google.com/github/machineGroup/image-super-resolution/blob/master/srcnn_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#change directory 
cd drive/My Drive

/content/drive/My Drive


In [0]:
from zipfile import ZipFile
file_name = "yang91.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("done");

In [0]:
#delete the zip data after extract
#!rm -rf yang91.zip

In [5]:
# import the necessary packages
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense, Flatten
from keras.optimizers import Adam
from skimage.measure import compare_ssim as ssim
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
import os

# python magic function, displays pyplot figures in the notebook
%matplotlib inline

Using TensorFlow backend.


In [0]:
# define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
         
    # assume RGB image
    target_data = target.astype(float)
    ref_data = ref.astype(float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    # the MSE between the two images is the sum of the squared difference between the two images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True))
    
    return scores

In [0]:
import os
import cv2
import h5py
import numpy
def read_training_data(file):
    with h5py.File(file, 'r') as hf:
        data = numpy.array(hf.get('data'))
        label = numpy.array(hf.get('label'))
        train_data = numpy.transpose(data, (0, 2, 3, 1))
        train_label = numpy.transpose(label, (0, 2, 3, 1))
        return train_data, train_label


In [0]:
# prepare degraded images by introducing quality distortions via resizing

def prepare_images(path, factor):
    
    # loop through the files in the directory
    for file in os.listdir(path):
        
        # open the file
        img = cv2.imread(path + '/' + file)
        
        # find old and new image dimensions
        h, w, _ = img.shape
        new_height =int( h / factor )
        new_width =int( w / factor )
        
        # resize the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        
        # resize the image - up
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        
        # save the image
        print('Saving {}'.format(file))
        cv2.imwrite('images/{}'.format(file), img)

In [9]:
prepare_images('source', 3)

Saving t1.bmp
Saving t13.bmp
Saving t14.bmp
Saving t15.bmp
Saving t16.bmp
Saving t17.bmp
Saving t18.bmp
Saving t19.bmp
Saving t2.bmp
Saving t20.bmp
Saving t21.bmp
Saving t22.bmp
Saving t24.bmp
Saving t25.bmp
Saving t26.bmp
Saving t27.bmp
Saving t28.bmp
Saving t29.bmp
Saving t3.bmp
Saving t30.bmp
Saving t32.bmp
Saving t34.bmp
Saving t35.bmp
Saving t36.bmp
Saving t37.bmp
Saving t38.bmp
Saving t39.bmp
Saving t4.bmp
Saving t40.bmp
Saving t42.bmp
Saving t43.bmp
Saving t44.bmp
Saving t45.bmp
Saving t46.bmp
Saving t47.bmp
Saving t48.bmp
Saving t49.bmp
Saving t5.bmp
Saving t50.bmp
Saving t51.bmp
Saving t52.bmp
Saving t53.bmp
Saving t54.bmp
Saving t58.bmp
Saving t59.bmp
Saving t60.bmp
Saving t61.bmp
Saving t62.bmp
Saving t63.bmp
Saving t65.bmp
Saving t66.bmp
Saving t7.bmp
Saving tt1.bmp
Saving tt13.bmp
Saving tt17.bmp
Saving tt19.bmp
Saving tt2.bmp
Saving tt20.bmp
Saving tt21.bmp
Saving tt24.bmp
Saving tt25.bmp
Saving tt26.bmp
Saving tt27.bmp
Saving tt3.bmp
Saving tt4.bmp
Saving tt5.bmp
Saving 

In [10]:
# test the generated images using the image quality metrics

for file in os.listdir('images/'):
    
    # open target and reference images
    target = cv2.imread('images/{}'.format(file))
    ref = cv2.imread('source/{}'.format(file))
    
    # calculate score
    scores = compare_images(target, ref)

    # print all three scores with new line characters (\n) 
    print('{}\nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(file, scores[0], scores[1], scores[2]))

t1.bmp
PSNR: 27.603695332967582
MSE: 338.71316912782646
SSIM: 0.8207098368750197

t13.bmp
PSNR: 30.35569584363294
MSE: 179.73474143339408
SSIM: 0.8648000371551077

t14.bmp
PSNR: 38.050615713668805
MSE: 30.559063733343752
SSIM: 0.9336349178236235

t15.bmp
PSNR: 29.15939674441448
MSE: 236.7347112923106
SSIM: 0.8371087115030488

t16.bmp
PSNR: 31.693474536821608
MSE: 132.08520114942527
SSIM: 0.8918964399303015

t17.bmp
PSNR: 25.35128150072508
MSE: 568.9491664862525
SSIM: 0.7767409145500573

t19.bmp
PSNR: 26.38306044464403
MSE: 448.6374992153663
SSIM: 0.8014631663261419

t18.bmp
PSNR: 22.50179871797101
MSE: 1096.5330948121646
SSIM: 0.771274927453535

t2.bmp
PSNR: 26.324832878555874
MSE: 454.69306223203847
SSIM: 0.8775230468099546

t24.bmp
PSNR: 30.744856035199906
MSE: 164.32968095391556
SSIM: 0.884815152595058

t20.bmp
PSNR: 25.345675252792553
MSE: 569.6840894148586
SSIM: 0.7616678876187931

t25.bmp
PSNR: 31.824616514315608
MSE: 128.1563011063011
SSIM: 0.9215844979664718

t21.bmp
PSNR: 30.3

In [0]:
# define the SRCNN model
def model():
    
    # define model type
    SRCNN = Sequential()
    
    # add model layers
    SRCNN.add(Conv2D(filters=128, kernel_size = (9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
    SRCNN.add(Conv2D(filters=64, kernel_size = (3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
    SRCNN.add(Conv2D(filters=1, kernel_size = (5, 5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
    
    # define optimizer
    adam = Adam(lr=0.0003)
    
    # compile model
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return SRCNN

In [0]:
# define necessary image processing functions

def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img


def shave(image, border):
    img = image[border: -border, border: -border]
    return img

In [13]:

X_train=[]
Y_train=[]
i=0

for file in os.listdir('images'):
    
    img_path='images/{}'.format(file)
    # load the degraded and reference images
    path, file = os.path.split(img_path)
    degraded = cv2.imread(img_path)
    ref = cv2.imread('source/{}'.format(file))
    
    # preprocess the image with modcrop
    ref = modcrop(ref, 3)
    degraded = modcrop(degraded, 3)
    
    # convert the image to YCrCb - (srcnn trained on Y channel)
    tempX = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    tempY = cv2.cvtColor(ref, cv2.COLOR_BGR2YCrCb)
    
    # create image slice and normalize  
    X = np.zeros((tempX.shape[0], tempX.shape[1], 1), dtype=float)
    X[:, :, 0] = tempX[:, :, 0].astype(float) / 255
    
    print(X.shape)
    
    
    Y = np.zeros((tempY.shape[0], tempY.shape[1], 1), dtype=float)
    Y[:, :, 0] = tempY[:, :, 0].astype(float) / 255
    
    
    
    X_train.append(X)
    Y_train.append(Y)
    
    
        
# Convert list to array
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(X_train.shape)
print(Y_train.shape)

(174, 194, 1)
(177, 185, 1)
(162, 155, 1)
(198, 209, 1)
(159, 173, 1)
(147, 152, 1)
(177, 176, 1)
(129, 140, 1)
(177, 170, 1)
(105, 113, 1)
(78, 77, 1)
(198, 209, 1)
(111, 122, 1)
(129, 146, 1)
(132, 149, 1)
(174, 239, 1)
(153, 188, 1)
(162, 185, 1)
(132, 215, 1)
(153, 188, 1)
(120, 179, 1)
(150, 134, 1)
(180, 227, 1)
(174, 212, 1)
(240, 428, 1)
(198, 332, 1)
(183, 362, 1)
(207, 311, 1)
(186, 278, 1)
(186, 224, 1)
(153, 248, 1)
(210, 314, 1)
(144, 272, 1)
(231, 506, 1)
(180, 245, 1)
(195, 242, 1)
(240, 332, 1)
(153, 149, 1)
(198, 422, 1)
(261, 329, 1)
(213, 323, 1)
(276, 284, 1)
(183, 275, 1)
(201, 218, 1)
(222, 317, 1)
(189, 245, 1)
(195, 392, 1)
(138, 263, 1)
(153, 179, 1)
(165, 236, 1)
(348, 386, 1)
(201, 242, 1)
(309, 314, 1)
(174, 212, 1)
(198, 314, 1)
(279, 284, 1)
(309, 410, 1)
(351, 434, 1)
(303, 371, 1)
(237, 371, 1)
(318, 431, 1)
(288, 410, 1)
(306, 305, 1)
(348, 434, 1)
(387, 365, 1)
(351, 371, 1)
(303, 443, 1)
(306, 410, 1)
(189, 248, 1)
(333, 434, 1)
(534, 1277, 1)
(543, 1

In [14]:
print(X_train[0].shape)
X_train = np.expand_dims(X_train, axis=0)
print(X_train.shape)

(174, 194, 1)
(1, 75)


In [15]:
srcnn = model()
srcnn.summary()



Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 128)   10496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 1)     1601      
Total params: 85,889
Trainable params: 85,889
Non-trainable params: 0
_________________________________________________________________


In [0]:
srcnn.fit(X_train, Y_train, batch_size = 32, epochs = 30)

In [0]:
#visulization 
from graphviz import Digraph
from keras.utils.vis_utils import plot_model
plot_model(srcnn, to_file='model_plot.png', show_shapes=True, show_layer_names=True)